In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
!pip install textblob


StatementMeta(, ec250bb0-440a-408d-b7ce-241190c57e2e, 3, Finished, Available)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 25.8 MB/s eta 0:00:00


In [2]:
# Step 1: Install necessary library
!pip install textblob

# Step 2: Import necessary libraries
from textblob import TextBlob
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import pyspark.sql.functions as F

# Step 3: Load your data
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

# Step 4: Define a function to determine sentiment
def analyze_sentiment(description):
    try:
        blob = TextBlob(description)
        sentiment = blob.sentiment.polarity
        if sentiment > 0:
            return 'Positive'
        elif sentiment < 0:
            return 'Negative'
        else:
            return 'Neutral'
    except:
        return 'Neutral'

# Register the function as a UDF
analyze_sentiment_udf = udf(analyze_sentiment, StringType())

# Step 5: Apply the function to the DataFrame
df_with_sentiment = df.withColumn('sentiment', analyze_sentiment_udf(F.col('description')))

# Display the new DataFrame with the sentiment column
display(df_with_sentiment)


StatementMeta(, ec250bb0-440a-408d-b7ce-241190c57e2e, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, ffd7b277-d8ee-4c07-80bb-12b95b9eaa99)

SynapseWidget(Synapse.DataFrame, bace1510-a343-4d1b-9954-fe589a8683f7)

In [4]:
#Incremental load code 
from pyspark.sql.utils import AnalysisException

try:
    table_name ='bing_lake_db.tbl_sentiment_analysis'
    df_with_sentiment.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print("Table already exits")

    df_with_sentiment.createOrReplaceGlobalTempView("vw_df_with_sentiment")

    spark.sql(f""" MERGE INTO {table_name} target_table
                   USING vw_df_with_sentimentl source view 

                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.image <> target_table.image OR
                   source view.provider <> target_table.provider OR  
                   source_view.datePublished < target_table.datePublished

                   THEN UPDATE SET *
                   WHEN NOT MATCHED THEN INSERT *

                """)

StatementMeta(, ec250bb0-440a-408d-b7ce-241190c57e2e, 6, Finished, Available)